# Getting started with Robust PCA

This notebook contains examples of how to solve Robust PCA problem.

For more details, please check the documentation website https://pygranso.readthedocs.io/en/latest/

1. Import all necessary modules and add PyGRANSO src folder to system path.

In [1]:
import time
import torch
import sys
## Adding PyGRANSO directories. Should be modified by user
sys.path.append('/home/buyun/Documents/GitHub/PyGRANSO')
from pygranso import pygranso
from pygransoStruct import Options, Data, GeneralStruct 

2. Specify torch device, and generate data

In [2]:
device = torch.device( 'cpu')
d1 = 5
d2 = 10
torch.manual_seed(1)
eta = .5
Y = torch.randn(d1,d2).to(device=device, dtype=torch.double)

3. Spceify optimization variables and corresponding objective and constrained function.

Note: please strictly follow the format of evalObjFunction and combinedFunction, which will be used in the PyGRANSO main algortihm. *X_struct* and *data_in* are always required.

In [3]:
# variables and corresponding dimensions.
var_in = {"M": [d1,d2],"S": [d1,d2]}

def evalObjFunction(X_struct):
    M = X_struct.M
    S = X_struct.S
    M.requires_grad_(True)
    S.requires_grad_(True)

    # objective function
    f = torch.norm(M, p = 'nuc') + eta * torch.norm(S, p = 1)
    return f

def combinedFunction(X_struct):
    M = X_struct.M
    S = X_struct.S
    M.requires_grad_(True)
    S.requires_grad_(True)
    
    # objective function
    f = torch.norm(M, p = 'nuc') + eta * torch.norm(S, p = 1)

    # inequality constraint, matrix form
    ci = None
    
    # equality constraint 
    ce = GeneralStruct()
    ce.c1 = M + S - Y

    return [f,ci,ce]

obj_eval_fn = lambda X_struct: evalObjFunction(X_struct)
comb_fn = lambda X_struct: combinedFunction(X_struct)

4. Specify user-defined options for PyGRANSO algorithm

In [4]:
opts = Options()
opts.QPsolver = 'osqp' 
opts.print_frequency = 10
opts.x0 = .2 * torch.ones((2*d1*d2,1)).to(device=device, dtype=torch.double)

4. Run main algorithm

In [5]:
start = time.time()
soln = pygranso(combinedFunction = comb_fn, objEvalFunction = obj_eval_fn,var_dim_map = var_in, torch_device = device, user_opts = opts)
end = time.time()
print("Total Wall Time: {}s".format(end - start))



╔═════ QP SOLVER NOTICE ══════════════════════════════════════════════════════════════╗
║  PyGRANSO requires a quadratic program (QP) solver that has a quadprog-compatible   ║
║  interface, as defined by osqp and Gurobi...                                        ║
╚═════════════════════════════════════════════════════════════════════════════════════╝
═════════════════════════════════════════════════════════════════════════════════════════════════════════════════╗
Problem specifications:                                                                                          ║ 
 # of variables                     :   100                                                                      ║ 
 # of inequality constraints        :     0                                                                      ║ 
 # of equality constraints          :    50                                                                      ║ 
═════╦═══════════════════════════╦════════════════╦════════════════

/home/buyun/anaconda3/envs/cuosqp_pygranso/lib/python3.9/site-packages/osqp/utils.py:122: UserWarning: Converting sparse A to a CSC (compressed sparse column) matrix. (It may take a while...)
  warn("Converting sparse A to a CSC " +


  10 ║ 0.150095 │  2.27183485513 ║  15.1360163268 ║   -  │ 4.53e-10 ║ S  │     2 │ 2.000000 ║     1 │ 0.117247   ║ 
  20 ║ 0.047101 │  0.70313740988 ║  14.9281994555 ║   -  │ 2.86e-10 ║ S  │     3 │ 0.250000 ║     1 │ 221.5967   ║ 
  30 ║ 0.008728 │  0.12961986324 ║  14.8511001075 ║   -  │ 2.55e-10 ║ S  │     1 │ 1.000000 ║     1 │ 0.008903   ║ 
  40 ║ 0.004638 │  0.06882630688 ║  14.8383798633 ║   -  │ 6.18e-12 ║ S  │     1 │ 1.000000 ║     1 │ 0.001264   ║ 
  50 ║ 0.001797 │  0.02666160637 ║  14.8366457710 ║   -  │ 1.03e-10 ║ S  │     4 │ 0.125000 ║     1 │ 0.002699   ║ 
  60 ║ 1.29e-04 │  0.00191396238 ║  14.8360805480 ║   -  │ 4.56e-10 ║ S  │    30 │ 1.86e-09 ║     1 │ 0.067763   ║ 
═════╩═══════════════════════════╩════════════════╩═════════════════╩═══════════════════════╩════════════════════╣
Optimization results:                                                                                            ║ 
F = final iterate, B = Best (to tolerance), MF = Most Feasible           